In [1]:
# import os
# import shutil
# import random
# from sklearn.model_selection import train_test_split

# bench_dir = 'bench_regression'
# if os.path.exists(bench_dir):
#     shutil.rmtree(bench_dir)
# os.mkdir(bench_dir)

# for i in os.listdir('datasets_chembl'):
#     with open(os.path.join('datasets_chembl', i), 'r') as f:
#         data = f.readlines()
#     data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
#     #
#     try:
#         data_train = random.sample(data_train, k=200)
#         data_test = random.sample(data_test, k=100)
#     except:
#         continue
#     #
#     data_folder = i.split('.')[0]
#     os.mkdir(os.path.join(bench_dir, data_folder))
#     #
#     with open(os.path.join(bench_dir, data_folder, 'train.csv'), 'w') as f:
#         for i in data_train:
#             f.write(i)
#     #
#     with open(os.path.join(bench_dir, data_folder, 'test.csv'), 'w') as f:
#         for i in data_test:
#             f.write(i)

In [2]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

import os 
import random
import shutil
import time

import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, balanced_accuracy_score

import pandas as pd

from rdkit import Chem

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

from sklearn.preprocessing import MinMaxScaler

from miprop.utils.logging import FailedMolecule, FailedConformer, FailedDescriptor

from miprop.utils.utils import df_to_list_of_bags

import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

## Benchmark datasets

In [3]:
def parse_data(data_path):
    data = pd.read_csv(data_path, header=None)
    
    mol_prop_list = []
    for smi, prop in zip(data[0], data[1]):
        mol = Chem.MolFromSmiles(smi)
        mol_prop_list.append((mol, prop))

    return mol_prop_list

In [4]:
BENCH_DATASETS = 'bench_regression'

## 2D single-instance models

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [ ]:
from molfeat.calc import (FPCalculator,
                          RDKitDescriptors2D, 
                          Pharmacophore2D, 
                          MordredDescriptors, 
                          CATS, 
                          ScaffoldKeyCalculator)

from molfeat.trans import MoleculeTransformer

In [ ]:
method_sil = [
              # linear models
              #(LinearRegression(), "LinearRegression"),
              (Ridge(), "RidgeRegression"), 
              (RandomForestRegressor(), "RandomForestRegressor"),
              (SVR(), "SVR"),
              (MLPRegressor(), "MLPRegressor"),
              ]

In [ ]:
descr_2d = [
            # fingerprints
            (FPCalculator("atompair"), "AtomPairBinary"),
            (FPCalculator("atompair-count"), "AtomPairCount"),
            (FPCalculator("avalon"), "AvalonBinary"),
            (FPCalculator("ecfp"), "ECFPBinary"),
            (FPCalculator("ecfp-count"), "ECFPCount"),
            (FPCalculator("erg"), "ERG"),
            (FPCalculator("estate"), "Estate"),
            (FPCalculator("fcfp"), "FCFPBinary"),
            (FPCalculator("fcfp-count"), "FCFPCount"),
            (FPCalculator("layered"), "Layered"),
            (FPCalculator("maccs"), "MACCS"),
            (FPCalculator("pattern"), "Pattern"),
            (FPCalculator("rdkit"), "RDKitBinary"),
            (FPCalculator("rdkit-count"), "RDKitCount"),
            (FPCalculator("secfp"), "SECFP"),
            (FPCalculator("topological"), "TopologicalBinary"),
            (FPCalculator("topological-count"), "TopologicalCount"),
            # RDKit
            (RDKitDescriptors2D(replace_nan=True), "RDKitDescriptors2D"),
            # Pmapper
            (Pharmacophore2D(replace_nan=True), "Pharmacophore2D"),
            # Mordred
            (MordredDescriptors(replace_nan=True), "MordredDescriptors"),
            # Scaffold
            (ScaffoldKeyCalculator(), "ScaffoldKey"),
           ]

In [ ]:
print(f'Total number of single-instance methods: {len(method_sil)}')
print(f'Total number of 2D descriptors: {len(descr_2d)}')
print(f'Total number of 2D models: {len(descr_2d) * len(method_sil)}')
#
res_folder = 'bench_results'
if os.path.exists(res_folder):
    shutil.rmtree(res_folder)
os.mkdir(res_folder)
#
for dataset in tqdm(os.listdir(BENCH_DATASETS)[:]):
    res = pd.DataFrame()
    #
    mols_train = parse_data(os.path.join(BENCH_DATASETS, dataset, 'train.csv'))
    mols_test = parse_data(os.path.join(BENCH_DATASETS, dataset, 'test.csv'))
    #
    res['TRUE'] = [i[1] for i in mols_test]

    # calc 2D descriptors
    for descr_func, descr_name in descr_2d:
        x_train = np.array(MoleculeTransformer(descr_func)([i[0] for i in mols_train]))
        x_test = np.array(MoleculeTransformer(descr_func)([i[0] for i in mols_test]))
        y_train = [i[1] for i in mols_train]
        y_test = [i[1] for i in mols_test]
        #
        scaler = MinMaxScaler()
        x_train_scaled = scaler.fit_transform(x_train)
        x_test_scaled = scaler.transform(x_test)
        #
        for method_func, method_name in method_sil:
            model = method_func.fit(x_train_scaled, y_train)
            y_pred = model.predict(x_test_scaled)
            #
            res[f'2D|None|{descr_name}|{method_name}'] = y_pred
            res.to_csv(os.path.join(res_folder, f'{dataset}_2D_SIL.csv'), index=False)

## 3D single-instance models

In [5]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [6]:
from miprop.mil.network.regressor import (AttentionNetworkRegressor,
                                          SelfAttentionNetworkRegressor,
                                          GatedAttentionNetworkRegressor,
                                          TemperatureAttentionNetworkRegressor,
                                          GumbelAttentionNetworkRegressor,
                                          GlobalTemperatureAttentionNetworkRegressor,
                                          DynamicPoolingNetworkRegressor,
                                          GaussianPoolingNetworkRegressor,
                                          InstanceNetworkRegressor,
                                          BagNetworkRegressor)

In [7]:
from miprop.conformer.rdkit import RDKitConformerGenerator

from miprop.descriptor.descriptor_3d.rdkit import (RDKitGEOM, 
                                                   RDKitAUTOCORR, 
                                                   RDKitMORSE, 
                                                   RDKitGETAWAY, 
                                                   RDKitRDF, 
                                                   RDKitWHIM)

from miprop.descriptor.descriptor_3d.molfeat import (MolFeatPharmacophore, 
                                                     MolFeatUSRD, 
                                                     MolFeatElectroShape)
from miprop.utils.scaler import BagMinMaxScaler

In [8]:
# sil methods
method_sil = [
              # linear models
              #(LinearRegression(), "LinearRegression"),
              (Ridge(), "RidgeRegression"), 
              (RandomForestRegressor(), "RandomForestRegressor"),
              (SVR(), "SVR"),
              (MLPRegressor(), "MLPRegressor"),
              ]

# mil methods
hparams = {'hidden_layer_sizes':(256, 128, 64),
           'num_epoch':300,
           'instance_weight_dropout':0.01}

method_mil = [
              (AttentionNetworkRegressor(**hparams), "AttentionNetworkRegressor"),
              (SelfAttentionNetworkRegressor(**hparams), "SelfAttentionNetworkRegressor"),
              (GatedAttentionNetworkRegressor(**hparams), "GatedAttentionNetworkRegressor"),
              (TemperatureAttentionNetworkRegressor(**hparams), "TemperatureAttentionNetworkRegressor"),
              (GumbelAttentionNetworkRegressor(**hparams), "GumbelAttentionNetworkRegressor"),
              (GlobalTemperatureAttentionNetworkRegressor(**hparams), "GlobalTemperatureAttentionNetworkRegressor"),
              (DynamicPoolingNetworkRegressor(**hparams), "DynamicPoolingNetworkRegressor"),
              (GaussianPoolingNetworkRegressor(**hparams), "GaussianPoolingNetworkRegressor"),
              (InstanceNetworkRegressor(**hparams), "InstanceNetworkRegressor"),
              (BagNetworkRegressor(**hparams), "BagNetworkRegressor")
]

In [9]:
descr_3d = [
            # RDKit
            (RDKitGEOM(), "RDKitGEOM"),
            (RDKitAUTOCORR(), "RDKitAUTOCORR"),
            (RDKitMORSE(), "RDKitMORSE"),
            (RDKitGETAWAY(), "RDKitGETAWAY"),
            (RDKitRDF(), "RDKitRDF"),
            (RDKitWHIM(), "RDKitWHIM"),
            # MolFeat
            (MolFeatPharmacophore(), "MolFeatPmapper"),
            (MolFeatUSRD(), "MolFeatUSRD"),
            (MolFeatElectroShape(), "MolFeatElectroShape")
            ]

In [13]:
print(f'Total number of single-instance methods: {len(method_sil)}')
print(f'Total number of 3D descriptors: {len(descr_3d)}')
print(f'Total number of 3D models: {len(descr_3d) * len(method_sil)}')
#
res_folder = 'bench_results'
# if os.path.exists(res_folder):
#     shutil.rmtree(res_folder)
# os.mkdir(res_folder)
#
num_conf = 5
for dataset in tqdm(os.listdir(BENCH_DATASETS)[1:2]):
    res = pd.DataFrame()
    #
    mols_train = parse_data(os.path.join(BENCH_DATASETS, dataset, 'train.csv'))
    mols_test = parse_data(os.path.join(BENCH_DATASETS, dataset, 'test.csv'))
    #
    res['TRUE'] = [i[1] for i in mols_test]
    # gen conformers
    conf_gen = RDKitConformerGenerator(num_conf=num_conf)
    #
    confs_train, y_train = [], []
    for mol_prop_tuple in mols_train:
        mol = conf_gen.generate_conformers_for_molecules([mol_prop_tuple[0]])[0]
        if not isinstance(mol, FailedConformer):
            confs_train.append((mol, mol_prop_tuple[1]))
            y_train.append(mol_prop_tuple[1])
    
    # calc 3D descriptors
    for descr_func, descr_name in descr_3d:
        x_train = descr_func.transform([i[0] for i in confs_train])
        #
        scaler = BagMinMaxScaler()
        x_train_scaled = scaler.fit(x_train)
        x_train_scaled = scaler.transform(x_train)
        #
        if num_conf == 1:
            method_tmp = method_sil
            x_train_scaled = np.concatenate(x_train_scaled)
            res_file = os.path.join(res_folder, f'{dataset}_3D_SIL.csv')
        if num_conf > 1:
            method_tmp = method_mil
            res_file = os.path.join(res_folder, f'{dataset}_3D_MIL.csv')
        #
        for method_func, method_name in method_tmp:
            model = method_func.fit(x_train_scaled, y_train)
            #
            y_pred = []
            for mol_prop_tuple in mols_test:
                mol = conf_gen.generate_conformers_for_molecules([mol_prop_tuple[0]])[0]
                if isinstance(mol, FailedConformer):
                    y_pred.append(np.mean(y_train))
                else:
                    x_test = descr_func.transform([mol])
                    x_test_scaled = scaler.transform(x_test)
                    #
                    y_pred.append(model.predict(x_test_scaled[0]).item())
            #
            res[f'3D|{num_conf}|{descr_name}|{method_name}'] = y_pred
            res.to_csv(res_file, index=False)

Total number of single-instance methods: 4
Total number of 3D descriptors: 9
Total number of 3D single-instance models: 36


  0%|                 | 0/1 [01:14<?, ?it/s]


RuntimeError: The size of tensor a (11) must match the size of tensor b (5) at non-singleton dimension 1

In [ ]:
res_2d = pd.read_csv('bench_results/CHEMBL243_2D_SIL.csv', index_col='Unnamed: 0')
res_sil = pd.read_csv('bench_results/CHEMBL243_3D_SIL.csv', index_col='Unnamed: 0')
#
res_mil = pd.read_csv('bench_results/CHEMBL243_3D_MIL.csv', index_col='Unnamed: 0')
res_mil = res_mil.dropna(axis=1, how='all')
#
res = pd.concat([res_2d,
                 res_sil[res_sil.columns[1:]],
                 res_mil[res_mil.columns[1:]]], axis=1)

In [ ]:
stat = pd.DataFrame()

for model in res.columns[1:]:
    mae = mean_absolute_error(res['TRUE'], res[model])
    r2 = r2_score(res['TRUE'], res[model])
    
    stat.loc[model, 'MAE'] = mae
    stat.loc[model, 'R2'] = r2
#
stat = stat.round(2)
stat = stat.sort_values(by='R2', ascending=False)
stat.head(20)